In [68]:
import pandas as pd
import glob
import datetime as dt

def flex_parse(ts: str) -> dt.datetime:
    # alles ab dem ersten Komma wegschneiden
    ts_clean = ts.split(',', 1)[0]
    for fmt in ("%d.%m.%Y %H:%M:%S", "%Y-%m-%d %H:%M:%S"):
        try:
            return dt.datetime.strptime(ts_clean, fmt)
        except ValueError:
            continue
    # Fallback
    return pd.to_datetime(ts_clean, dayfirst=True, errors='raise')

general_info_keys = [
    'measurement_category',
    'measurement_unit', 'measurement_type', 'measurement_value_type',
    'measurement_frequency', 'location_id', 'category', 'usage',
    'usage_detail', 'country', 'federal_state', 'city', 'post_code',
    'area', 'construction_year'
]

csv_files = glob.glob(
    r'C:\Users\jonas\Desktop\Studium\Master\SS 2025\Anwendungsfelder Business Analytics\Daten_Intervall\2024\*.csv'
)

all_dfs = []

for file in csv_files:
    # 1) Metadaten einlesen (Zeilen 0–17)
    df_meta = pd.read_csv(
        file,
        sep=';',
        header=None,
        names=['field', 'value'],
        dtype=str,
        decimal=',',
        nrows=18
    )
    # Whitespace weg
    df_meta['field'] = df_meta['field'].str.strip()
    df_meta['value'] = df_meta['value'].str.strip()

    # Dict mit allen Keys (None, falls nicht gefunden)
    general_info = {
        key: df_meta.loc[df_meta['field']==key, 'value'].squeeze()
             if key in df_meta['field'].values else None
        for key in general_info_keys
    }

    # 2) Messdaten ab Zeile 23 einlesen
    meas = pd.read_csv(
        file,
        sep=';',
        skiprows=22,
        header=None,
        names=['Zeitstempel', 'Messwert'],
        dtype={'Zeitstempel': str},
        decimal=',',
        thousands='.'
    )

    # 3) Zeitstempel parsen
    meas['Zeitstempel'] = meas['Zeitstempel'].apply(flex_parse)

    # 4) Messwert in Float
    meas['Messwert'] = pd.to_numeric(meas['Messwert'], errors='raise')

    # 5) Metadaten anfügen
    for key, val in general_info.items():
        meas[key] = val

    meas['area'] = pd.to_numeric(meas['area'], errors='raise')
    meas['construction_year'] = pd.to_numeric(meas['construction_year'], errors='raise')
    meas["category"] = meas["category"].astype("category")
    meas["measurement_value_type"] = meas["measurement_value_type"].astype("category")
    meas["measurement_frequency"] = meas["measurement_frequency"].astype("category")
    meas["measurement_type"] = meas["measurement_type"].astype("category")
    meas["measurement_unit"] = meas["measurement_unit"].astype("category")
    meas["country"] = meas["country"].astype("category")
    meas["federal_state"] = meas["federal_state"].astype("category")

    all_dfs.append(meas)

# 6) Alles zusammenführen
final_df_2024 = pd.concat(all_dfs, ignore_index=True)

In [77]:
pdList = [final_df, final_df_2020, final_df_2021, final_df_2022, final_df_2023, final_df_2024]
complete_df = pd.concat(pdList, ignore_index=True)

In [ ]:
def optimize_object_columns(df, convert_numeric=False):
    initial_memory = df.memory_usage(deep=True).sum() / 1024 ** 2

    for col in df.select_dtypes(include=['object']).columns:
        num_unique_values = df[col].nunique()
        num_total_values = len(df[col])
        
        # Wenn viele Wiederholungen, konvertiere zu category
        if num_unique_values / num_total_values < 0.5:
            df[col] = df[col].astype('category')
        elif convert_numeric:
            # Versuche Konvertierung zu float/int (z. B. "123" → 123)
            try:
                df[col] = pd.to_numeric(df[col])
            except (ValueError, TypeError):
                df[col] = df[col].astype('string')
        else:
            df[col] = df[col].astype('string')

    optimized_memory = df.memory_usage(deep=True).sum() / 1024 ** 2
    print(f"Speicher vor Optimierung: {initial_memory:.2f} MB")
    print(f"Speicher nach Optimierung: {optimized_memory:.2f} MB")
    print(f"Reduziert um: {(initial_memory - optimized_memory):.2f} MB")

    return df

complete_df = optimize_object_columns(complete_df, convert_numeric=True)

In [80]:
print(complete_df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149291167 entries, 0 to 149291166
Data columns (total 17 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Zeitstempel             datetime64[ns]
 1   Messwert                float64       
 2   measurement_category    category      
 3   measurement_unit        category      
 4   measurement_type        category      
 5   measurement_value_type  category      
 6   measurement_frequency   category      
 7   location_id             category      
 8   category                category      
 9   usage                   category      
 10  usage_detail            float64       
 11  country                 category      
 12  federal_state           category      
 13  city                    category      
 14  post_code               category      
 15  area                    float64       
 16  construction_year       float64       
dtypes: category(12), datetime64[ns](1), float6

In [ ]:
final_df.to_csv("IntervallMessdaten2019.csv")
final_df_2020.to_csv("IntervallMessdaten2020.csv")
final_df_2021.to_csv("IntervallMessdaten2021.csv")
final_df_2022.to_csv("IntervallMessdaten2022.csv")
final_df_2023.to_csv("IntervallMessdaten2023.csv")
final_df_2024.to_csv("IntervallMessdaten2024.csv")
complete_df.to_csv("IntervallMessdaten2019-2024.csv")

In [109]:
from collections import defaultdict

# 1. Parameter
start = pd.Timestamp('2019-01-01')
end   = pd.Timestamp('2023-12-31')
expected_days = (end - start).days + 1  # 2041

# 2. Dictionary: sensor_id → Set von Tagen
sensor_days = defaultdict(set)

# 3. Chunk-Größe (z.B. 1 Mio Zeilen)
chunksize = 10**6
n = len(complete_df)

for i in range(0, n, chunksize):
    # a) Chunk aus dem vorhandenen df herausziehen (View, kein Copy der Datenblocks)
    sub = complete_df.iloc[i : min(i+chunksize, n)]
    
    # b) Auf Zeitraum filtern
    mask = (sub['Zeitstempel'] >= start) & (sub['Zeitstempel'] <= end)
    sub = sub.loc[mask]
    if sub.empty:
        continue
    
    # c) Auf Tagesauflösung casten — nur für den Chunk, spart RAM
    #    (8 Byte * chunksize statt 8 Byte * Gesamtzahl Zeilen)
    sub = sub.copy()  # wegen SettingWithCopy-Warnung
    sub['Datum'] = sub['Zeitstempel'].values.astype('datetime64[D]')
    
    # d) Duplicate Sensor-Datum-Kombi im Chunk entfernen
    sub = sub.drop_duplicates(subset=['location_id', 'Datum'])
    
    # e) Sets pro Sensor updaten
    for sid, day in zip(sub['location_id'], sub['Datum']):
        sensor_days[sid].add(day)

# 4. Sensoren herausfiltern, die an exakt allen 2041 Tagen gemessen haben
valid_sensors = [sid for sid, days in sensor_days.items() if len(days) == expected_days]
print(f"Gefundene Sensoren: {len(valid_sensors)}")

# 5. Den Original-df nach diesen Sensoren filtern
df_clean = complete_df[complete_df['location_id'].isin(valid_sensors)].reset_index(drop=True)

Gefundene Sensoren: 115


In [93]:
df_clean.to_csv("SensorenMitVollständigenDaten.csv")

In [94]:
df_wetterdaten = pd.read_csv("Wetterdaten.csv")
df_ferien = pd.read_csv("ferientage_2000_2025.csv")

df_feiertage = pd.read_excel("DeutscheFeiertage.xlsx", 
                             dtype={"Feiertag": "category", 
                                    "Datum": "object", 
                                    "Bundesland": "category"})

df_feiertage["Datum"] = pd.to_datetime(df_feiertage["Datum"]).dt.date

In [97]:
# Prüfung, ob Wetterdaten von mehreren Stationen im gleichen Bundesland pro Tag vorliegen
# Wie viele Duplikate gibt es pro (date, state) in den Wetterdaten?
dups = df_wetterdaten.groupby(['date', 'state']).size().reset_index(name='count')
dups = dups[dups['count'] > 1]
print(dups)

                             date            state  count
1       2000-01-01 00:00:00+00:00           Bayern      7
2       2000-01-01 00:00:00+00:00           Berlin      4
3       2000-01-01 00:00:00+00:00      Brandenburg      9
4       2000-01-01 00:00:00+00:00           Bremen      9
5       2000-01-01 00:00:00+00:00          Hamburg      9
...                           ...              ...    ...
117804  2024-12-31 00:00:00+00:00  Rheinland-Pfalz      5
117805  2024-12-31 00:00:00+00:00         Saarland      9
117806  2024-12-31 00:00:00+00:00          Sachsen      5
117807  2024-12-31 00:00:00+00:00   Sachsen-Anhalt      5
117808  2024-12-31 00:00:00+00:00        Thüringen      8

[117371 rows x 3 columns]


# Sensordaten anreichern

In [ ]:
# Sicherstellen, dass die Datumsformate übereinstimmen
df_clean['Datum_Messung'] = pd.to_datetime(df_clean['Zeitstempel']).dt.date
df_wetterdaten['date'] = pd.to_datetime(df_wetterdaten['date']).dt.date

agg_dict = {
    'station_id': 'first',
    'resolution': 'first',
    'dataset': 'first',
    'parameter': 'first',
    'value': 'mean',        # numerisch
    'quality': 'min',
    'name': 'first'
}

# Wetterdaten vorher pivotisieren, da sonst Duplikate nach dem Join im DataFrame auftreten
df_wetter_pivot = df_wetterdaten.pivot_table(
    index=['date', 'state'],
    columns='parameter',
    values='value',
    aggfunc='mean'
).reset_index()

# Klimadaten mergen anhand Datum und Bundesland
df = pd.merge(
    df_clean,
    df_wetter_pivot,
    left_on=['Datum_Messung', 'federal_state'],
    right_on=['date', 'state'],
    how='left'
)

# doppelte Zeit- und Ortsangaben entfernen
df.drop(columns=['date', 'state'], inplace=True)

# nur die Messwerte beibehalten, die auch Klimainformationen haben
# df = df.loc[df["temperature_air_mean_2m"].isna() == False]

# Wochentag
df["Wochentag"] = pd.to_datetime(df["Zeitstempel"]).apply(lambda x: dt.date.isoweekday(x))

# in Bezeichnung umschlüsseln
dict_weekday = {1: "Montag",
                2: "Dienstag",
                3: "Mittwoch",
                4: "Donnerstag",
                5: "Freitag",
                6: "Samstag",
                7: "Sonntag"}

df["Wochentag_Name"] = df["Wochentag"].apply(lambda x: dict_weekday.get(x))

# Feiertage mergen
df = pd.merge(
    df,
    df_feiertage,
    left_on=['Datum_Messung', 'federal_state'],
    right_on=['Datum', 'Bundesland'],
    how='left'
)

df.drop(columns=['Bundesland'], inplace=True)

# selbes Datumformat zum mergen, hier gab es vorher Probleme ohne Konvertierung des Datums
df["Datum_Messung"] = pd.to_datetime(df["Datum_Messung"])
df_ferien["Datum"] = pd.to_datetime(df_ferien["Datum"])

# Ferien mergen
df = pd.merge(
    df,
    df_ferien,
    left_on=["Datum_Messung", "federal_state"],
    right_on=["Datum", "Bundesland"],
    how="left"
)

In [111]:
df.head(30)

,Zeitstempel,Messwert,measurement_category,measurement_unit,measurement_type,measurement_value_type,measurement_frequency,location_id,category,usage,...,temperature_air_mean_2m,wind_gust_max,wind_speed,Wochentag,Wochentag_Name,Feiertag,Datum_x,Datum_y,Bundesland,Ferientyp
0,2019-01-01 00:15:00,0.082590,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
1,2019-01-01 00:30:00,0.046686,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
2,2019-01-01 00:45:00,0.036041,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
3,2019-01-01 01:00:00,0.042784,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
4,2019-01-01 01:15:00,0.037057,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
5,2019-01-01 01:30:00,0.030309,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
6,2019-01-01 01:45:00,0.033961,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
7,2019-01-01 02:00:00,0.051470,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
8,2019-01-01 02:15:00,0.042131,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten
9,2019-01-01 02:30:00,0.046302,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,4.8,NaN,NaN,2,Dienstag,Neujahr,2019-01-01,2019-01-01,Bayern,Weihnachten


In [112]:
df.drop(columns=["Datum_x", "Datum_y"], inplace=True)

In [113]:
df.head()

,Zeitstempel,Messwert,measurement_category,measurement_unit,measurement_type,measurement_value_type,measurement_frequency,location_id,category,usage,...,snow_depth,sunshine_duration,temperature_air_mean_2m,wind_gust_max,wind_speed,Wochentag,Wochentag_Name,Feiertag,Bundesland,Ferientyp
0,2019-01-01 00:15:00,0.082590,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,0.0,0.0,4.8,NaN,NaN,2,Dienstag,Neujahr,Bayern,Weihnachten
1,2019-01-01 00:30:00,0.046686,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,0.0,0.0,4.8,NaN,NaN,2,Dienstag,Neujahr,Bayern,Weihnachten
2,2019-01-01 00:45:00,0.036041,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,0.0,0.0,4.8,NaN,NaN,2,Dienstag,Neujahr,Bayern,Weihnachten
3,2019-01-01 01:00:00,0.042784,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,0.0,0.0,4.8,NaN,NaN,2,Dienstag,Neujahr,Bayern,Weihnachten
4,2019-01-01 01:15:00,0.037057,Verbrauch,kWh,Wirkarbeit,Zaehlerstand,15min,0079edf0-c6be-4945-b1aa-f8f985952f68,Privat,Haushalt,...,0.0,0.0,4.8,NaN,NaN,2,Dienstag,Neujahr,Bayern,Weihnachten


In [114]:
df.to_csv("SensorenMitVollständigenDaten2019-2023.csv")